In [1]:
import numpy as np
import pandas as pd
from collections import Counter

def entropy(data):
    counts = Counter(data)
    total = len(data)
    entropy_value = -sum((count / total) * np.log2(count / total) for count in counts.values())
    return entropy_value

def information_gain(data, feature_index, target):
    total_entropy = entropy(target)
    values, counts = np.unique(data[:, feature_index], return_counts=True)
    weighted_entropy = sum(
        (counts[i] / len(target)) * entropy(target[data[:, feature_index] == values[i]])
        for i in range(len(values))
    )
    return total_entropy - weighted_entropy

def id3(data, target, features, depth=0, max_depth=None):
    if len(np.unique(target)) == 1:
        return target[0]
    if len(features) == 0 or (max_depth is not None and depth >= max_depth):
        return Counter(target).most_common(1)[0][0]
    feature_gains = [information_gain(data, i, target) for i in range(len(features))]
    best_feature_index = np.argmax(feature_gains)
    best_feature = features[best_feature_index]
    tree = {best_feature: {}}
    values = np.unique(data[:, best_feature_index])
    for value in values:
        sub_data = data[data[:, best_feature_index] == value]
        sub_target = target[data[:, best_feature_index] == value]
        sub_features = features[:best_feature_index] + features[best_feature_index + 1:]
        subtree = id3(
            np.delete(sub_data, best_feature_index, axis=1),
            sub_target,
            sub_features,
            depth=depth + 1,
            max_depth=max_depth
        )
        tree[best_feature][value] = subtree
    return tree

def print_tree(tree, depth=0):
    if isinstance(tree, dict):
        for key, value in tree.items():
            print("  " * depth + str(key))
            print_tree(value, depth + 1)
    else:
        print("  " * depth + f"--> {tree}")

if __name__ == "__main__":
    data = np.array([
        ['Sunny', 'Hot', 'High', 'Weak', 'No'],
        ['Sunny', 'Hot', 'High', 'Strong', 'No'],
        ['Overcast', 'Hot', 'High', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Cool', 'Normal', 'Strong', 'No'],
        ['Overcast', 'Cool', 'Normal', 'Strong', 'Yes'],
        ['Sunny', 'Mild', 'High', 'Weak', 'No'],
        ['Sunny', 'Cool', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'Normal', 'Weak', 'Yes'],
        ['Sunny', 'Mild', 'Normal', 'Strong', 'Yes'],
        ['Overcast', 'Mild', 'High', 'Strong', 'Yes'],
        ['Overcast', 'Hot', 'Normal', 'Weak', 'Yes'],
        ['Rain', 'Mild', 'High', 'Strong', 'No']
    ])
    feature_names = ['Outlook', 'Temperature', 'Humidity', 'Wind']
    target = data[:, -1]
    features = feature_names
    tree = id3(data[:, :-1], target, features)
    print("Structured Decision Tree:")
    print_tree(tree)


Structured Decision Tree:
Outlook
  Overcast
    --> Yes
  Rain
    Wind
      Strong
        --> No
      Weak
        --> Yes
  Sunny
    Humidity
      High
        --> No
      Normal
        --> Yes
